<a href="https://colab.research.google.com/github/kamrulhasan04947/mlprac/blob/main/deepHybridLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm

In [ ]:
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import numpy as np
import seaborn as sns
from keras.applications.vgg16 import VGG16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SIZE =244
train_data_dir = '/content/drive/MyDrive/dataset/satelitedata/train/'
test_data_dir = '/content/drive/MyDrive/dataset/satelitedata/test/'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    rotation_range = 180,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    validation_split = 0.2)

train_generator = data_generator.flow_from_directory(
    train_data_dir,
    target_size =(SIZE, SIZE),
    batch_size = 32,
    shuffle = True,
    class_mode = 'categorical',
    seed = 42,
    subset='training')

validation_generator = data_generator.flow_from_directory(
    test_data_dir,
    target_size =(SIZE, SIZE),
    batch_size = 32,
    shuffle = True,
    class_mode = 'categorical',
    seed = 42,
    subset='validation')

Found 4800 images belonging to 4 classes.
Found 320 images belonging to 4 classes.


In [ ]:
import tqdm
train_generator.reset()
x_train, y_train = next(train_generator)
for i in tqdm.tqdm(range(int(train_generator.n/32)-1)):
  img, label = next(train_generator)
  x_train = np.append(x_train, img, axis=0 )
  y_train = np.append(y_train, label, axis=0)
print(x_train.shape, y_train.shape)

100%|██████████| 149/149 [19:00<00:00,  7.66s/it]

(4800, 244, 244, 3) (4800, 4)


In [ ]:
import tqdm
validation_generator.reset()
x_test, y_test = next(validation_generator)
for i in tqdm.tqdm(range(int(validation_generator.n/32)-1)):
  img, label = next(validation_generator)
  x_test = np.append(x_test, img, axis=0 )
  y_test = np.append(y_test, label, axis=0)
print(x_test.shape, y_test.shape)

100%|██████████| 9/9 [01:03<00:00,  7.09s/it]

(320, 244, 244, 3) (320, 4)


In [ ]:
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
VGG_model.input

<KerasTensor: shape=(None, 244, 244, 3) dtype=float32 (created by layer 'input_1')>

In [ ]:
x = Flatten()(VGG_model.output)
prediction = Dense(4, activation='softmax')(x)
model = Model(inputs=VGG_model.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0     

In [ ]:
feature_extractor=VGG_model.predict(x_train)

150/150 [==============================] - 3606s 24s/step


In [ ]:
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
dt = DecisionTreeClassifier(criterion='entropy', random_state=10)
modelfit = dt.fit(features, y_train)
prediction = modelfit.predict(x_test)
report = classification_report(y_test, prediction)

ValueError: ignored

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

import tensorflow as tf

from keras.models import load_model

model.save('facefeatures_new_model.h5')

Epoch 1/5
 18/188 [=>............................] - ETA: 3:56:33 - loss: 1.7729 - accuracy: 0.2875